In [1]:
%%bash

# JVM needs to be re-installed
apt-get update > /dev/null
apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Descargar Hadoop
wget -q https://downloads.apache.org/hadoop/common/hadoop-3.3.5/hadoop-3.3.5.tar.gz
tar -xzf hadoop-3.3.5.tar.gz
mv hadoop-3.3.5/ /usr/local/

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [2]:
import os

HADOOP_VERSION = "3.3.5"
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"   # default is changed
os.environ["HADOOP_HOME"] = f"/usr/local/hadoop-{HADOOP_VERSION}/"

# Agrega el Hadoop BIN al PATH
current_path = os.getenv('PATH')
new_path = current_path + f":/usr/local/hadoop-{HADOOP_VERSION}/bin/"
os.environ["PATH"] = new_path

# Verificar si PATH contiene Hadoop
!echo $PATH

/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/usr/local/hadoop-3.3.5/bin/


In [3]:
!hadoop version

Hadoop 3.3.5
Source code repository https://github.com/apache/hadoop.git -r 706d88266abcee09ed78fbaa0ad5f74d818ab0e9
Compiled by stevel on 2023-03-15T15:56Z
Compiled with protoc 3.7.1
From source with checksum 6bbd9afcf4838a0eb12a5f189e9bd7
This command was run using /usr/local/hadoop-3.3.5/share/hadoop/common/hadoop-common-3.3.5.jar


In [4]:
!mkdir -p /usr/local/hadoop/etc/hadoop/

In [5]:
!wget -q https://downloads.apache.org/hive/hive-3.1.3/apache-hive-3.1.3-bin.tar.gz
!tar xzf apache-hive-3.1.3-bin.tar.gz

In [6]:
os.environ["HIVE_HOME"] = "/content/apache-hive-3.1.3-bin"

!echo $HIVE_HOME

/content/apache-hive-3.1.3-bin


In [7]:
current_path = os.getenv('PATH')
new_path = current_path+':/content/apache-hive-3.1.3-bin/bin'
os.environ["PATH"] = new_path
!echo $PATH

/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/usr/local/hadoop-3.3.5/bin/:/content/apache-hive-3.1.3-bin/bin


In [8]:
!echo $JAVA_HOME
!echo $HADOOP_HOME
!echo $HIVE_HOME

/usr/lib/jvm/java-8-openjdk-amd64
/usr/local/hadoop-3.3.5/
/content/apache-hive-3.1.3-bin


In [9]:
!rm *.tar.gz

In [10]:
!hdfs dfs -mkdir /tmp
!hdfs dfs -chmod g+w /tmp

!hdfs dfs -mkdir -p /content/warehouse
!hdfs dfs -chmod g+w /content/warehouse

mkdir: `/tmp': File exists


In [11]:
!ls $HADOOP_HOME/share/hadoop/common/lib/*slf4j*
!ls $HIVE_HOME/lib/*slf4j*

/usr/local/hadoop-3.3.5//share/hadoop/common/lib/jul-to-slf4j-1.7.36.jar
/usr/local/hadoop-3.3.5//share/hadoop/common/lib/slf4j-api-1.7.36.jar
/usr/local/hadoop-3.3.5//share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar
/content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar


In [12]:
!mv /content/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar ./

In [13]:
!ls $HIVE_HOME/lib/gu*

/content/apache-hive-3.1.3-bin/lib/guava-19.0.jar


In [14]:
!ls $HADOOP_HOME/share/hadoop/hdfs/lib/gu*

/usr/local/hadoop-3.3.5//share/hadoop/hdfs/lib/guava-27.0-jre.jar


In [15]:
!mv $HIVE_HOME/lib/guava-19.0.jar ./
!cp $HADOOP_HOME/share/hadoop/hdfs/lib/guava-27.0-jre.jar $HIVE_HOME/lib/

In [16]:
# Crea un directorio en el sistema de archivos distribuido de Hadoop (HDFS)
# La opción '-p' asegura que se creen todos los directorios padres necesarios
# En este caso, el directorio '/data/' se creará si no existe ya
!hadoop fs -mkdir -p /data/

In [17]:
# Descargar Hadoop versión 3.3.1
!wget https://archive.apache.org/dist/hadoop/common/hadoop-3.3.1/hadoop-3.3.1.tar.gz

# Extraer el archivo descargado
!tar -xzf hadoop-3.3.1.tar.gz

--2024-09-16 20:37:42--  https://archive.apache.org/dist/hadoop/common/hadoop-3.3.1/hadoop-3.3.1.tar.gz
Resolving archive.apache.org (archive.apache.org)... 65.108.204.189, 2a01:4f9:1a:a084::2
Connecting to archive.apache.org (archive.apache.org)|65.108.204.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 605187279 (577M) [application/x-gzip]
Saving to: ‘hadoop-3.3.1.tar.gz’

hadoop-3.3.1.tar.gz 100%[===================>] 577.15M  17.3MB/s    in 34s     

2024-09-16 20:38:17 (16.7 MB/s) - ‘hadoop-3.3.1.tar.gz’ saved [605187279/605187279]



In [18]:
!hadoop-3.3.1/share/hadoop/tools/lib/hadoop-streaming-3.3.1.jar

/bin/bash: line 1: hadoop-3.3.1/share/hadoop/tools/lib/hadoop-streaming-3.3.1.jar: Permission denied


In [19]:
# Extraer el archivo tar.gz
!tar -xzf hadoop-3.3.1.tar.gz

# Cantidad total de vuelos que existieron en cada día del mes. Denomina al maper como maper_1.py.

In [25]:
%%writefile /content/mapper_1.py
import sys

def main():
    """Función principal del mapper que procesa entradas de vuelos."""
    for line in sys.stdin:
        # Dividir la línea por comas
        fields = line.strip().split(',')

        # Ignorar la línea de encabezado
        if fields[0] == 'Year':
            continue

        # Extraer el día del mes
        day_of_month = fields[3]  # Columna "DayofMonth"

        # Emitir el día del mes con un conteo de 1
        print(f"{day_of_month}\t1")

if __name__ == "__main__":
    main()

Overwriting /content/mapper_1.py


# Cantidad total de vuelos que existieron en cada día del mes. Denomina al reducer como reducer_1.py.

In [26]:
%%writefile /content/reducer_1.py
import sys

def main():
    """Función principal del reducer que suma el conteo de vuelos por día."""
    current_day = None
    total_flights = 0

    for line in sys.stdin:
        line = line.strip()
        day_of_month, count = line.split('\t')
        count = int(count)

        if current_day == day_of_month:
            total_flights += count
        else:
            if current_day is not None:
                print(f"{current_day}\t{total_flights}")
            current_day = day_of_month
            total_flights = count

    if current_day is not None:
        print(f"{current_day}\t{total_flights}")

if __name__ == "__main__":
    main()

Overwriting /content/reducer_1.py


In [28]:
# Asigna permisos de ejecución a los archivos mapper_1.py y reducer_1.py en el directorio /content/.
!chmod +x /content/mapper_1.py
!chmod +x /content/reducer_1.py

In [38]:
# Ejecuta un trabajo de Hadoop Streaming usando el archivo hadoop-streaming-3.3.1.jar.
# Utiliza mapper_1.py y reducer_1.py como scripts de mapeo y reducción, respectivamente.
# Procesa el archivo de entrada /data/2024_3.csv y guarda los resultados en /data/total_flights_day_month.
!hadoop jar hadoop-3.3.1/share/hadoop/tools/lib/hadoop-streaming-3.3.1.jar \
    -files /content/mapper_1.py,/content/reducer_1.py \
    -mapper "python3 /content/mapper_1.py" \
    -reducer "python3 /content/reducer_1.py" \
    -input /data/2024_3.csv \
    -output /data/total_flights_day_month

2024-09-16 21:03:02,100 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2024-09-16 21:03:02,281 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2024-09-16 21:03:02,281 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2024-09-16 21:03:02,315 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2024-09-16 21:03:02,843 INFO mapred.FileInputFormat: Total input files to process : 1
2024-09-16 21:03:02,872 INFO mapreduce.JobSubmitter: number of splits:8
2024-09-16 21:03:03,199 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local95009450_0001
2024-09-16 21:03:03,199 INFO mapreduce.JobSubmitter: Executing with tokens: []
2024-09-16 21:03:03,727 INFO mapred.LocalDistributedCacheManager: Localized file:/content/mapper_1.py as file:/tmp/hadoop-root/mapred/local/job_local95009450_0001_75a1a7c8-e023-4072-a295-9d0dcec0e020/mapper_1.py
2024-09-16 21:03:03,766 INFO mapred.LocalDistributedCacheMa

In [39]:
# Lista los archivos y directorios en /data/total_flights_day_month en HDFS.
!hdfs dfs -ls /data/total_flights_day_month

Found 2 items
-rw-r--r--   1 root root          0 2024-09-16 21:03 /data/total_flights_day_month/_SUCCESS
-rw-r--r--   1 root root        285 2024-09-16 21:03 /data/total_flights_day_month/part-00000


In [40]:
# Muestra el contenido de todos los archivos que comienzan con 'part-' en el directorio /data/total_flights_day_month en HDFS.
!hdfs dfs -cat /data/total_flights_day_month/part-*

"DayofMonth"	1
1	19157
10	19623
11	19977
12	18541
13	18804
14	20107
15	20120
16	17720
17	19828
18	20102
19	18604
2	16492
20	18862
21	20130
22	20132
23	17718
24	19802
25	20096
26	18601
27	18863
28	20129
29	20134
3	19079
30	17714
31	19691
4	19085
5	17603
6	17967
7	19830
8	19882
9	17374


# Conteo total de vuelos que salieron de los diferentes orígenes (campo OriginAirportID). Denomina al maper como maper_2.py.

In [31]:
%%writefile /content/mapper_2.py

import sys

def mapper():
    """Función mapper que procesa entradas de datos de vuelos y emite el ID del aeropuerto de origen."""
    for line in sys.stdin:
        # Eliminar espacios en blanco y dividir la línea en campos
        line = line.strip()
        fields = line.split(',')

        # Asegurarse de que la línea tenga suficientes campos
        if len(fields) > 15:  # Ajusta según el número total de campos en tus datos
            origin_airport_id = fields[12]  # Suponiendo que OriginAirportID es el campo en la posición 12
            if origin_airport_id.isdigit():
                # Emitir la clave y el valor
                print(f"{origin_airport_id}\t1")

if __name__ == "__main__":
    mapper()

Writing /content/mapper_2.py


# Conteo total de vuelos que salieron de los diferentes orígenes (campo OriginAirportID). Denomina al reducer como reducer_2.py.

In [32]:
%%writefile /content/reducer_2.py

import sys

def reducer():
    """Función reducer que suma el conteo de vuelos por ID de aeropuerto."""
    current_airport_id = None
    total_flights = 0

    for line in sys.stdin:
        line = line.strip()
        if not line:
            continue

        airport_id, count = line.split("\t")
        count = int(count)

        if current_airport_id == airport_id:
            total_flights += count
        else:
            if current_airport_id is not None:
                # Imprimir el resultado para el aeropuerto anterior
                print(f"{current_airport_id}\t{total_flights}")
            current_airport_id = airport_id
            total_flights = count

    # Imprimir el resultado para el último aeropuerto
    if current_airport_id is not None:
        print(f"{current_airport_id}\t{total_flights}")

if __name__ == "__main__":
    reducer()

Writing /content/reducer_2.py


In [34]:
# Asigna permisos de ejecución a los archivos mapper_2.py y reducer_2.py en el directorio /content/.
!chmod +x /content/mapper_2.py
!chmod +x /content/reducer_2.py

In [35]:
# Ejecuta un trabajo de Hadoop Streaming usando el archivo hadoop-streaming-3.3.1.jar.
# El trabajo usa mapper_2.py y reducer_2.py como scripts de mapeo y reducción, respectivamente.
# Procesa el archivo de entrada /data/2024_3.csv y guarda los resultados en /data/total_flights_airport_id.
!hadoop jar hadoop-3.3.1/share/hadoop/tools/lib/hadoop-streaming-3.3.1.jar \
    -files /content/mapper_2.py,/content/reducer_2.py \
    -mapper "python3 /content/mapper_2.py" \
    -reducer "python3 /content/reducer_2.py" \
    -input /data/2024_3.csv \
    -output /data/total_flights_airport_id

2024-09-16 20:59:05,008 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2024-09-16 20:59:05,260 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2024-09-16 20:59:05,261 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2024-09-16 20:59:05,298 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2024-09-16 20:59:05,901 INFO mapred.FileInputFormat: Total input files to process : 1
2024-09-16 20:59:05,928 INFO mapreduce.JobSubmitter: number of splits:8
2024-09-16 20:59:06,255 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local1705715392_0001
2024-09-16 20:59:06,255 INFO mapreduce.JobSubmitter: Executing with tokens: []
2024-09-16 20:59:06,834 INFO mapred.LocalDistributedCacheManager: Localized file:/content/mapper_2.py as file:/tmp/hadoop-root/mapred/local/job_local1705715392_0001_6056310b-2a8e-446f-985e-7246b7bdb288/mapper_2.py
2024-09-16 20:59:06,856 INFO mapred.LocalDistributedCac

In [36]:
# Muestra el contenido del archivo /data/total_flights_airport_id/part-00000 en HDFS, que es parte de la salida del trabajo de Hadoop Streaming.
!hdfs dfs -cat /data/total_flights_airport_id/part-00000

1013506	392
1013603	147
1014005	1893
1014106	62
1014602	62
1015502	93
1015706	148
1015804	327
1016506	9
1017004	60
1018502	155
1020803	252
1025702	1031
1027506	62
1027903	427
1029906	1302
1033302	54
1037205	1041
1039707	28459
1040805	530
1042302	7154
1043106	862
1043405	117
1046602	611
1046902	88
1052907	1757
1055102	62
1055803	54
1056103	187
1057705	31
1058102	257
1059904	1372
1061705	58
1062003	276
1062702	292
1063104	62
1066602	255
1067603	94
1068503	142
1069302	8287
1071302	1723
1072102	11324
1072805	62
1073103	62
1073203	245
1073907	53
1074706	189
1075405	31
1077902	58
1078105	373
1078502	479
1079206	1838
1080003	2254
1082106	8076
1084907	878
1086803	523
1087402	270
1091802	53
1092603	62
1098002	574
1099005	253
1099402	2053
1100303	646
1101303	62
1102705	18
1104205	3415
1104903	93
1105703	17400
1106606	3461
1106703	124
1107603	62
1110902	972
1111106	147
1112205	31
1114008	336
1114608	274
1115005	88
1119302	3558
1120302	62
1123302	57
1125203	258
1125904	6272
1126702	656
1127805	119

In [37]:
# Crea el directorio /data/Resultados, incluyendo cualquier directorio intermedio necesario.
!mkdir -p /data/Resultados

# Resultados en tabla, respectivamente "total_flights_day_month.csv".

In [42]:
#!/usr/bin/env python3
import pandas as pd

def main():
    """Lee el archivo de resultados y guarda el CSV con encabezados en la carpeta de resultados."""
    # Leer el archivo de resultados sin encabezados
    df = pd.read_csv('/data/total_flights_day_month/part-00000', header=None,
                     names=['Day', 'TotalFlights'])

    # Guardar el archivo CSV con encabezados en la carpeta "data/Resultados"
    df.to_csv('/data/Resultados/total_flights_day_month.csv', index=False)

if __name__ == "__main__":
    main()

# Resultados en tabla, respectivamente "total_flights_airport_id.csv".

In [43]:
#!/usr/bin/env python3
import pandas as pd

def main():
    """Lee el archivo de resultados sin encabezados y guarda el CSV con encabezados en la carpeta de resultados."""
    # Leer el archivo de resultados sin encabezados
    df = pd.read_csv('/data/total_flights_airport_id/part-00000', header=None,
                     names=['OriginAirportID', 'TotalFlights'])

    # Guardar el archivo CSV con encabezados en la carpeta "data/Resultados"
    df.to_csv('/data/Resultados/total_flights_airport_id.csv', index=False)

if __name__ == "__main__":
    main()